In [1]:
from nvidia.dali.pipeline import Pipeline
from nvidia.dali import pipeline_def
import nvidia.dali.fn as fn
import nvidia.dali.types as types
from nvidia.dali.plugin.pytorch import DALIGenericIterator

import torchvision.models as models
resnet18 = models.resnet18()

from time import time

In [3]:
data_dir = "augraphy_cache"
@pipeline_def(batch_size=128, num_threads=4, device_id=0)
def simple_mixed_pipeline():
    jpegs, labels = fn.readers.file(file_root=data_dir)
    images = fn.decoders.image(jpegs, device="mixed")

    return images, labels


pipe = simple_mixed_pipeline(batch_size=8, num_threads=2, device_id=0)
start = time()
pipe.build()
pipe_out = pipe.run()
end = time()
print(pipe_out)


/data/ephemeral/home/baseline_code/.conda/lib/python3.11/site-packages/nvidia/dali/backend.py:99: Warning: nvidia-dali-cuda120 is no longer shipped with CUDA runtime. You need to install it separately. cuFFT is typically provided with CUDA Toolkit installation or an appropriate wheel. Please check https://docs.nvidia.com/cuda/cuda-quick-start-guide/index.html#pip-wheels-installation-linux for the reference.
  deprecation_warning(
/data/ephemeral/home/baseline_code/.conda/lib/python3.11/site-packages/nvidia/dali/backend.py:110: Warning: nvidia-dali-cuda120 is no longer shipped with CUDA runtime. You need to install it separately. NPP is typically provided with CUDA Toolkit installation or an appropriate wheel. Please check https://docs.nvidia.com/cuda/cuda-quick-start-guide/index.html#pip-wheels-installation-linux for the reference.
  deprecation_warning(
/data/ephemeral/home/baseline_code/.conda/lib/python3.11/site-packages/nvidia/dali/backend.py:121: Warning: nvidia-dali-cuda120 is no

RuntimeError: Critical error when building pipeline:
Error in CPU operator `nvidia.dali.fn.readers.file`,
which was used in the pipeline definition with the following traceback:

  File "/tmp/ipykernel_566431/1977029101.py", line 4, in simple_mixed_pipeline
    jpegs, labels = fn.readers.file(file_root=data_dir)

encountered:

Assert on "SizeImpl() > 0" failed: No files found.
C++ context: [/opt/dali/dali/operators/reader/loader/file_label_loader.h:180] 
Current pipeline object is no longer valid.

# 앞으로 돌려봐야할 것

In [ ]:


data_dir = "./test_DALI/images/"


@pipeline_def(batch_size=128, num_threads=4, device_id=0)
def get_dali_pipeline(data_dir, crop_size):
  images, labels = fn.readers.file(file_root=data_dir)
  # decode data on the GPU
  images = fn.decoders.image_random_crop(images, device="mixed", output_type=types.RGB)
  # the rest of processing happens on the GPU as well
  images = fn.resize(images, resize_x=crop_size, resize_y=crop_size)
  images = fn.crop_mirror_normalize(images,
                                    mean=[0.485 * 255,0.456 * 255,0.406 * 255],
                                    std=[0.229 * 255,0.224 * 255,0.225 * 255],
                                    mirror=fn.random.coin_flip())
  return images, label


train_data = DALIGenericIterator(
   [get_dali_pipeline(data_dir, (224, 224))], ['data', 'label'], reader_name='Reader')


for i, data in enumerate(train_data):
  x, y = data[0]['data'], data[0]['label']
  pred = resnet18(x)
  # loss = loss_func(pred, y)
  # backward(loss, model)